# Install dependencies for the Biobakery pipeline (if not already installed)

```
conda create -n Biobakery python=3.7 -y
conda activate Biobakery
pip install metaphlan
pip install humann
```


# Declare variables and binac function
## Always run it before other cells in the notebook

In [1]:
### VARIABLES ###

# Provide variables here
WS = '/beegfs/work/graaf20/Rstc2020' # working space for this project
DB = '/beegfs/work/graaf20/Databases/' # path for databases
OUT = 'Biobakery' # biobakery outdir 
MPV = 'MetaPhlan_4_6'
HMV = 'Humann_3_7'
CONDA = '$HOME/miniconda3/etc/profile.d/conda.sh' # change if different
LOGIN = 'ho_graaf20' # your login in Binac

### Do not change from here ###
RR = 'Raw_reads/'
CR = 'Pooled/'
STATS = 'Reports/'

##################### DO NOT CHANGE ################################
import os
import pandas as pd

params = '''
#PBS -l nodes={nodes}:ppn={ppn}
#PBS -l walltime={time}
#PBS -l mem={mem}gb
#PBS -N {name}.sh
#PBS -q {queue}
#PBS -S /bin/bash
export TMPDIR=$TMPDIR
''' 

source = f'source {CONDA}'

VARS = (
f'WS={WS}\n'
f'RR={RR}\n'
f'CR={CR}\n'
f'DB={DB}\n'
f'MPV={MPV}\n'
f'HMV={HMV}\n'
f'STATS={STATS}\n'
f'OUT={OUT}\n')

COMM = '#Custom variables and commands\n{todo}'
STRING = '\n'.join([params, source, VARS, COMM])

# binac function that will submit the job to server
def binac(nodes = 1, ppn = 1, time = '00:15:00', mem = 2, name = 'myjob',
          queue = 'tiny', todo = 'echo "Command"', STRING = STRING):
    text = STRING
    if '{nodes}' in STRING:
        text = STRING.format(nodes = nodes, ppn = ppn, time = time,
               mem = mem, name = name, queue = queue, todo = todo)
    script = f'{name}.sh'
    with open(script, "w") as text_file:
        text_file.write(text)
    !chmod +x $script
    !qsub $script
    

# Create list of file indexes to access it later
clean = !rm -r */.ipynb_checkpoints
samples = []
if os.path.exists(CR): 
    samples = [f.split('_')[0] for f in os.listdir(CR) if any(['fq' in f,'fastq' in f])]
    samples = sorted(list(set(samples)))
print('Number of clean samples: ', len(samples))
if samples != []:
    print('List of clean samples:')
    print(*samples)

Number of clean samples:  52
List of clean samples:
1 100 102 104 106 108 11 110 123 124 125 126 127 128 129 13 130 131 132 133 134 15 17 19 21 23 25 27 29 31 4 44 45 46 47 48 49 50 51 52 53 54 55 80 83 88 9 90 92 94 96 98


# Biobakery

## Install databases

In [17]:
!rm -rf ../Databases/Humann3_6 ../Databases/MetaPhlan4

In [19]:
# Download databases. Skip if already done

# Nothing to change from here
todo = '''
conda activate Biobakery

cd $DB
mkdir $MPV $HMV

echo "Downloading MetaPhlan databases"

metaphlan --install --bowtie2db $MPV/

echo "Done"
echo "Downloading Humann chocophlan databases"

humann_databases --download chocophlan full $HMV/chocophlan --update-config yes

echo "Done"
echo "Downloading Humann uniref databases"

humann_databases --download uniref uniref90_diamond $HMV/uniref90 --update-config yes

echo "Done"
echo "Downloading Humann mapping databases"

humann_databases --download utility_mapping full $HMV/mapping --update-config yes

echo "Done."
'''

binac(ppn=4, time='10:00:00', mem=64, name=f'Biobakery_DB', queue='short', todo=todo)

10947390


In [24]:
# Clean jupyter lab backup folders that can mess with jobs
clean = !rm -r ../Databases/*/.ipynb_checkpoints ../Databases/*/*/.ipynb_checkpoints

In [ ]:
#keep output reports 

!mkdir $STATS/DB_Reports
!mv Biobakery_db.sh.* $STATS/DB_Reports/

## Run MetaPhlan4 and Humann3.7

In [2]:
# Run Biobakery

### Important! ###
# Sometimes jobs are finished abnormally due to unknown reasons (I suspect 
# it is related somehow to the stars). So just run this cell again when all
# jobs are finished (!). It will check output directories, delete outputs 
# for failed samples and submit them again. If nothing happened (no jobs 
# submitted, no text printed) then you are good and all samples worked as 
# supposed. Stars are on your side today!

# Attention! Before running it for all your samples, run it only for one sample.
test = False # if True then test with 1 sample. If False will start all unprocessed samples.

# Nothing to change from here
todo = '''
conda activate Biobakery

##### Define variables #####
name={name}
mindex={mindex}
ppn={ppn}

echo "Creating output directories..."

cd $WS
mkdir -p $OUT/MetaPhlan $OUT/Humann/$name

##### Concatenate reads for Biobakery #####
echo "Concatenating reads for Biobakery..."

fq=${{name}}.fastq.gz
cat $CR/${{name}}_*_1.fastq.gz $CR/${{name}}_*_2.fastq.gz > $OUT/$fq

echo "Done."
echo
echo "############################################################"
echo "###                  Running Metaphlan4                  ###"
echo "############################################################"
echo
echo "Check if taxonomical profile already exists..."

cd $OUT
profile=MetaPhlan/${{name}}_profiled_metagenome.txt
if [ -f "$profile" ]; then
    echo "$profile exists. Skipping MetaPhlan..."
else 
    echo "$profile does not exist. Starting MetaPhlan..."
    metaphlan $fq --bowtie2out MetaPhlan/${{name}}_bowtie2.bz2 \
    --nproc $ppn --input_type fastq --index $mindex \
    --bowtie2db $DB/$MPV -t rel_ab_w_read_stats -o $profile
fi

echo "Done."
echo
echo "############################################################"
echo "###                   Running Humann3                    ###"
echo "############################################################"
echo

humann --input $fq --output Humann/$name --remove-temp-output --threads $ppn \
--metaphlan-options "--index $mindex --bowtie2db $DB/$MPV" --resume

echo "Done."
echo
### Clean ###
echo "Cleaning..."

rm $fq

echo "Done."
echo
echo "############################################################"
echo "###                   All done! Enjoy!                   ###"
echo "### Citations:                                           ###"
echo "### 1. BioBakery3:                                       ###"
echo "###    https://elifesciences.org/articles/65088          ###"
echo "############################################################"
'''

mindex = [i for i in os.listdir(f'{DB}/{MPV}') if i.endswith('.pkl')][0].strip('.pkl')
reports = f'{STATS}/Biobakery_Reports'
ready = []

!mkdir -p $reports 

# Check outputs (if there are any)
check = f'{OUT}/Humann/'
if os.path.exists(check):
    for out in os.listdir(f'{OUT}/Humann/'):
        if out.startswith('.'):
            continue
        sub = f'{OUT}/Humann/{out}'
        if ' '.join(os.listdir(sub)).count('.tsv') == 3:
            ready.append(out)
            for file in os.listdir(sub):
                if '.tsv' not in file:
                    !rm -rf $sub/$file

failed = [r.split('_Biob')[0] for r in os.listdir('./') if r.endswith('_feedback')]
failed = [r for r in failed if r not in ready]
submitted = [r.split('_Biob')[0] for r in os.listdir('./') if r.endswith('.sh')]
submitted = [r for r in submitted if r not in ready + failed]

#launch unprocessed samples
for read in samples:
    ppn = 2
    shs = f'{read}_Biobakery.sh*'
    
    #test with the first read only 
    if test and read != samples[0]: 
        continue
    
    #check if ready or running
    if read in ready:
        m = !mv $shs $reports/
        continue
    
    #check if submitted
    if read in submitted:
        continue

    #Check if humann run failed for some samples. Then clean and rerun.
    if read in failed:
        print(f'Sample {read} does not finished correctly. Attempting to rerun...')
        r = !rm -rf $shs

    print(read)
    do = todo.format(name = read, ppn = ppn, mindex = mindex)
    binac(ppn=ppn, time='07:00:00:00', mem=180, name=f'{read}_Biobakery', queue='smp', todo=do)
    
print(f'{len(ready)} samples are ready with a {len(submitted)} more well on the way')
print(f'Total amount of samples: {len(samples)}')

52 samples are ready with a 0 more well on the way
Total amount of samples: 52


In [3]:
!qstat -u ho_graaf20


mgmt02: 
                                                                                  Req'd       Req'd       Elap
Job ID                  Username    Queue    Jobname          SessID  NDS   TSK   Memory      Time    S   Time
----------------------- ----------- -------- ---------------- ------ ----- ------ --------- --------- - ---------
10948186                ho_graaf20  smp      SM_reannotate_MA  28222     1      8     164gb 696:00:00 R 150:13:32


In [138]:
!qdel 10947877

In [ ]:
jobs = !qstat -u $LOGIN
jobs = jobs[5:]
running = [l.split(' ')[0] for l in jobs if ' R ' in l]
queued = [l.split(' ')[0] for l in jobs if ' Q ' in l]
queued

['10800012',
 '10800013',
 '10800014',
 '10800015',
 '10800016',
 '10800017',
 '10800018',
 '10800019',
 '10800020',
 '10800021',
 '10800022',
 '10800023',
 '10800024',
 '10800025',
 '10800026',
 '10800027',
 '10800028',
 '10800029',
 '10800030',
 '10800031',
 '10800032',
 '10800033',
 '10800034',
 '10800035',
 '10800036',
 '10800037',
 '10800038',
 '10800039',
 '10800040',
 '10800041',
 '10800042',
 '10800043',
 '10800044',
 '10800045',
 '10800046',
 '10800047',
 '10800048',
 '10800049',
 '10800050',
 '10800051',
 '10800515',
 '10802428',
 '10802429',
 '10802430',
 '10802431',
 '10802432',
 '10802433',
 '10802434',
 '10802435',
 '10802436',
 '10802437',
 '10802438',
 '10802439',
 '10802440',
 '10802441',
 '10802442',
 '10802443',
 '10802444',
 '10802445',
 '10802446',
 '10802447',
 '10802448',
 '10802449',
 '10802450',
 '10802451',
 '10802452',
 '10802453',
 '10802454',
 '10802455',
 '10802456',
 '10802457',
 '10802458',
 '10802459',
 '10802460',
 '10802461',
 '10802462',
 '10802463',

In [ ]:
for job in queued:
    !qalter -l walltime=07:00:00:00 $job

In [24]:
with open('Running_jobs.txt', 'w') as f:
    for job in running:
        f.write(f"{job}\n")

In [61]:
!wget -O merge_metaphlan_tables_abs.py \
https://github.com/timyerg/Metaphlan-absolute-abundance-merger/blob/main/merge_metaphlan4_tables_abs.py?raw=true

--2022-12-02 15:29:51--  https://github.com/timyerg/Metaphlan-absolute-abundance-merger/blob/main/merge_metaphlan4_tables_abs.py?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/timyerg/Metaphlan-absolute-abundance-merger/raw/main/merge_metaphlan4_tables_abs.py [following]
--2022-12-02 15:29:51--  https://github.com/timyerg/Metaphlan-absolute-abundance-merger/raw/main/merge_metaphlan4_tables_abs.py
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/timyerg/Metaphlan-absolute-abundance-merger/main/merge_metaphlan4_tables_abs.py [following]
--2022-12-02 15:29:51--  https://raw.githubusercontent.com/timyerg/Metaphlan-absolute-abundance-merger/main/merge_metaphlan4_tables_abs.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 

### Merge MetaPhlan tables

In [4]:
todo = '''
conda activate Biobakery

cd $WS/$OUT

echo "Creating output directory..."

out=Merged_tables

mkdir -p $out

##### Merge Metaphlan tables #####
echo "Merging Metaphlan relative abundances..."

merge_metaphlan_tables.py MetaPhlan/*_profiled_metagenome.txt > $out/Metaphlan_rel_abund_table.tsv

echo "Merging Metaphlan absolute abundances..."

wget -O merge_metaphlan_tables_abs.py \
https://github.com/timyerg/Metaphlan-absolute-abundance-merger\
/blob/main/merge_metaphlan4_tables_abs.py?raw=true

chmod +x merge_metaphlan_tables_abs.py

./merge_metaphlan_tables_abs.py MetaPhlan/*_profiled_metagenome.txt > $out/Metaphlan_abs_abund_table.tsv

echo "Done."

rm merge_metaphlan_tables_abs.py
'''

binac(time='15:00', mem=6, name=f'Merge_metaphlan', queue='tiny', todo=todo.format())

10949165


### Merge Humann tables

In [4]:
todo = '''
conda activate Biobakery

out=Merged_tables

cd $WS/$OUT

echo "Creating output directory..."

mkdir -p $out

##### Merge Humann tables #####

mkdir temp
cp Humann/*/*.tsv temp/

echo "Merging Humann genefamilies..."
humann_join_tables -i temp -o $out/genefamilies_table.tsv --file_name genefamilies

echo "Merging Humann pathcoverage..."
humann_join_tables -i temp -o $out/pathcoverage_table.tsv --file_name pathcoverage

echo "Merging Humann pathabundance..."
humann_join_tables -i temp -o $out/pathabundance_table.tsv --file_name pathabundance

echo "Done."
rm -rf temp
'''

binac(time='04:00:00', mem=16, name=f'Merge_Humann', queue='short', todo=todo.format())

10950379


In [6]:
!mv Merge_Humann.sh* $STATS/Biobakery_Reports/

### Regroup

In [7]:
todo = '''
conda activate Biobakery

out=Merged_tables

cd $WS/$OUT

##### Regroup tables #####
groups=("rxn" "pfam" "ko" "go")

for group in ${{groups[@]}}; do
    echo "Regrouping genefamilies_table.tsv to $group..."
    humann_regroup_table -i $out/genefamilies_table.tsv -g uniref90_${{group}} \
    -o $out/${{group}}_genefamilies_table.tsv
done

echo "Done."
'''

binac(time='04:00:00', mem=96, name=f'Regroup_tables', queue='short', todo=todo.format())

10950380


In [2]:
!mv Regroup_tables.sh* $STATS/Biobakery_Reports/

### Normalize humann outputs

In [9]:
todo = '''
conda activate Biobakery

out=Merged_tables

cd $WS/$OUT/$out

### Normalize Humann tables ###
norm=("relab" "cpm")
for path in ./*_table.tsv; do
    tab="${{path##*/}}"
    if [[ "$tab" == *'genefamilies'* || "$tab" == *'pathabundance'* ]]; then
        for n in ${{norm[@]}}; do
            echo "Normalize $tab to $n..."
            humann_renorm_table -i $tab -o ${{n}}_${{tab}} -u $n -p
        done
    fi
done
    
echo "Done."
'''

binac(time='04:00:00', mem=124, name=f'Normalize_tables', queue='short', todo=todo.format())

10950381


In [12]:
!mv Normalize_tables.sh* $STATS/Biobakery_Reports/

### Add names to humann outputs

In [17]:
todo = '''
conda activate Biobakery

out=Merged_tables

cd $WS/$OUT/$out

### Add names to Humann tables ###

echo "########## Adding names to KO tables ##########"
names=("kegg-orthology")
for path in ./*_ko_genefamilies_table.tsv; do
    tab="${{path##*/}}"
    for name in ${{names[@]}}; do
        echo "Adding $name names to $tab..."
        humann_rename_table -i $tab -n $name -o ${{name}}_${{tab}}
    done
done

#echo "########## Adding names to GO tables ##########"
#names=("go")
#for path in ./*_go_genefamilies_table.tsv; do
#    tab="${{path##*/}}"
#    for name in ${{names[@]}}; do
#        echo "Adding $name names to $tab..."
#        humann_rename_table -i $tab -n $name -o ${{name}}_${{tab}}
#    done
#done

#echo "########## Adding names to MetaCyc tables ##########"
#names=("metacyc-rxn")
#for path in ./*_rxn_genefamilies_table.tsv; do
#    tab="${{path##*/}}"
#    for name in ${{names[@]}}; do
#        echo "Adding $name names to $tab..."
#        humann_rename_table -i $tab -n $name -o ${{name}}_${{tab}}
#    done
#done

#echo "########## Adding names to PFAM tables ##########"
#names=("pfam")
#for path in ./*_pfam_genefamilies_table.tsv; do
#    tab="${{path##*/}}"
#    for name in ${{names[@]}}; do
#        echo "Adding $name names to $tab..."
#        humann_rename_table -i $tab -n $name -o ${{name}}_${{tab}}
#    done
#done

echo "Done."
'''

binac(mem=8, name=f'Rename_tables', todo=todo.format())

10812745


In [20]:
todo = '''
conda activate Biobakery

out=Merged_tables

cd $WS/$OUT/$out

### Add names to Humann tables ###

names=("uniref90")
for path in ./*genefamilies_table.tsv; do
   tab="${{path##*/}}"
    for name in ${{names[@]}}; do
        echo "Adding $name names to $tab..."
        humann_rename_table -i $tab -n $name -o ${{name}}_${{tab}}
    done
done

echo "Done."
'''

binac(mem=32, time='04:00:00', queue='short', name=f'Rename_tables', todo=todo.format())

10950384


In [26]:
!mv Rename_tables.sh* $STATS/Biobakery_Reports/

In [27]:
# Zip biobakery folder to download it later
todo = '''
cd $WS
zip -r ${OUT}4.zip $OUT MetaPhlan4.ipynb
'''

binac(time='06:00:00', mem=8, name=f'Biobakrery4_zip', queue='short', todo=todo)

10950385


In [21]:
todo = '''
conda activate qiime2-2022.8

cd $WS

#Create matrix
qiime deicode rpca \
    --i-table core-metrics/uniref90_genefamilies_destratified.qza \
    --o-distance-matrix core-metrics/aitchison_distance_matrix.qza \
    --o-biplot core-metrics/aitchison_biplot.qza

#Create PCoA
qiime diversity pcoa \
    --i-distance-matrix core-metrics/aitchison_distance_matrix.qza \
    --o-pcoa core-metrics/aitchison_pcoa_results.qza

#Create PCoA plot
qiime emperor plot \
    --i-pcoa core-metrics/aitchison_pcoa_results.qza \
    --m-metadata-file core-metrics/metadata.tsv \
    --o-visualization core-metrics/aitchison_emperor.qzv
'''

binac(mem=250, time='04:00:00', queue='smp', name=f'Deicode', todo=todo)

10849992


In [8]:
!qdel 10950380

In [28]:
!qstat -u ho_graaf20


mgmt02: 
                                                                                  Req'd       Req'd       Elap
Job ID                  Username    Queue    Jobname          SessID  NDS   TSK   Memory      Time    S   Time
----------------------- ----------- -------- ---------------- ------ ----- ------ --------- --------- - ---------
10948186                ho_graaf20  smp      SM_reannotate_MA  28222     1      8     164gb 696:00:00 R 151:00:01
10950384                ho_graaf20  short    Rename_tables.sh    398     1      1      32gb  04:00:00 C       -- 
10950385                ho_graaf20  short    Biobakrery4_zip.   1287     1      1       8gb  06:00:00 R  00:00:10


In [11]:
!qstat -q smp


server: mgmt.binac

Queue            Memory CPU Time Walltime Node  Run Que Lm  State
---------------- ------ -------- -------- ----  --- --- --  -----
smp                --      --       --      --   11  10 --   E R
                                               ----- -----
                                                  11    10


In [141]:
!gunzip Pooled/98_FR_d13_Trt3_HP2_1.fastq.gz

^C


In [84]:
!zcat Pooled/98_FR_d13_Trt3_HP2_1.fastq.gz > Pooled/98_FR_d13_Trt3_HP2_1.fastq


gzip: Pooled/98_FR_d13_Trt3_HP2_1.fastq.gz: unexpected end of file


In [140]:
!gzip -t Pooled/98_FR_d13_Trt3_HP2_1.fastq.gz

In [110]:
!scp meco-ThinkPad-P52@144.41.181.122:/media/meco/Windows/Ubuntu/Pooled/*.gz /beegfs/work/graaf20/Rstc2020/Pooled/

ssh: connect to host 144.41.181.122 port 22: Connection timed out


In [15]:
import pandas as pd
df = pd.read_csv('Biobakery/Merged_tables/genefamilies_table.tsv', sep='\t', engine='python')

df

,# Gene Family,100_Abundance-RPKs,102_Abundance-RPKs,104_Abundance-RPKs,106_Abundance-RPKs,108_Abundance-RPKs,110_Abundance-RPKs,11_Abundance-RPKs,123_Abundance-RPKs,124_Abundance-RPKs,...,55_Abundance-RPKs,80_Abundance-RPKs,83_Abundance-RPKs,88_Abundance-RPKs,90_Abundance-RPKs,92_Abundance-RPKs,94_Abundance-RPKs,96_Abundance-RPKs,98_Abundance-RPKs,9_Abundance-RPKs
0,UNMAPPED,33272500.0,4.110360e+07,43380396.0,38903932.0,41236807.0,43420225.0,52364200.0,55097999.0,55092735.0,...,57302901.0,63060947.0,64843650.0,34503840.0,39705714.0,43017431.0,40321465.0,43196245.0,43637811.0,39508842.0
1,UniRef90_A0A009DWL0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,UniRef90_A0A009DWL0|unclassified,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,UniRef90_A0A009E034,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,UniRef90_A0A009E034|unclassified,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2425452,UniRef90_Z9JZS4|unclassified,0.0,5.174653e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2425453,UniRef90_Z9K090,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2425454,UniRef90_Z9K090|unclassified,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2425455,UniRef90_Z9K6I6,0.0,2.938711e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
